In [ ]:
# docker run -p 8888:8888 -p 4040:4040 -v "%cd%:/home/jovyan/work" quay.io/jupyter/pyspark-notebook

In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [41]:
spark = SparkSession.builder.appName("SeriesTemporales").getOrCreate()
# Leer archivo Parquet
df = spark.read.parquet("../../data/transformed/madrid_data/parquet/")
df = df.orderBy(["fecha", "hora"])
# Mostrar datos
df.groupBy("anho").count().show()

+----+-----+
|anho|count|
+----+-----+
|2023| 8760|
|2022| 8760|
|2020| 8784|
|2021| 8760|
|2019| 8760|
|2024| 8784|
|2025| 1393|
+----+-----+



In [42]:
# Definir ventana para obtener las 3 horas anteriores
window_lag = Window.partitionBy("indicativo", "anho", "mes", "dia").orderBy("hora")

# Agregar columnas de las 3 horas previas
df = df.withColumn("humedad_hora_1", F.lag("humedad_media", 1).over(window_lag)) \
       .withColumn("humedad_hora_2", F.lag("humedad_media", 2).over(window_lag)) \
       .withColumn("humedad_hora_3", F.lag("humedad_media", 3).over(window_lag)) \
       .withColumn("temp_hora_1", F.lag("temperatura_media", 1).over(window_lag)) \
       .withColumn("temp_hora_2", F.lag("temperatura_media", 2).over(window_lag)) \
       .withColumn("temp_hora_3", F.lag("temperatura_media", 3).over(window_lag))

In [43]:
# Calcular temperatura general por hora, día y mes en todo el DataFrame
df_temp_stats = df.groupBy("hora", "dia", "mes").agg(
    F.avg("temperatura_media").alias("temp_media_general"),
    F.min("temperatura_media").alias("temp_min_general"),
    F.max("temperatura_media").alias("temp_max_general")
)
# Unir las estadísticas generales al DataFrame original
df = df.join(df_temp_stats, on=["hora", "dia", "mes"], how="left")

In [44]:
# Calcular temperatura general por hora, día y mes en todo el DataFrame
df_humidity_stats = df.groupBy("hora", "dia", "mes").agg(
    F.avg("humedad_media").alias("humedad_media_general"),
    F.min("humedad_media").alias("humedad_min_general"),
    F.max("humedad_media").alias("humedad_max_general")
)
# Unir las estadísticas generales al DataFrame original
df = df.join(df_humidity_stats, on=["hora", "dia", "mes"], how="left")

In [45]:
df.show(20, truncate=False)

+----+---+---+----------+-------------+-----------------+----------+----+--------------+--------------+--------------+-----------+-----------+-----------+------------------+----------------+----------------+---------------------+-------------------+-------------------+
|hora|dia|mes|fecha     |humedad_media|temperatura_media|indicativo|anho|humedad_hora_1|humedad_hora_2|humedad_hora_3|temp_hora_1|temp_hora_2|temp_hora_3|temp_media_general|temp_min_general|temp_max_general|humedad_media_general|humedad_min_general|humedad_max_general|
+----+---+---+----------+-------------+-----------------+----------+----+--------------+--------------+--------------+-----------+-----------+-----------+------------------+----------------+----------------+---------------------+-------------------+-------------------+
|0   |1  |1  |2019-01-01|50.0         |3.1              |102       |2019|NULL          |NULL          |NULL          |NULL       |NULL       |NULL       |3.4285714285714284|0.6             |

In [46]:
from statsmodels.tsa.arima.model import ARIMA

In [47]:
df_pandas = df.toPandas()

In [48]:
# Variables exógenas
exog_variables = ['temp_hora_1', 'temp_hora_2', 'temp_hora_3', 'temp_media_general', 'temp_min_general', 'temp_max_general']

# Eliminar valores nulos o rellenar
df_pandas_clean = df_pandas.dropna(subset=exog_variables + ['temperatura_media'])
X = df_pandas_clean[exog_variables]
y = df_pandas_clean['temperatura_media']

# Asegurarse de que los datos sean numéricos
X = X.astype(float)
y = y.astype(float)

# Ajustar el modelo ARIMAX
arimax_model = ARIMA(y, exog=X, order=(2, 0, 1))
arimax_result = arimax_model.fit()

# Resumen del modelo
print(arimax_result.summary())

# Realizar pronósticos (usando las últimas filas de las variables exógenas)
forecast = arimax_result.forecast(steps=10, exog=X.tail(10))
print(forecast)

/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Usin

                               SARIMAX Results                                
Dep. Variable:      temperatura_media   No. Observations:                47250
Model:                 ARIMA(2, 0, 1)   Log Likelihood              -87261.507
Date:                Sat, 08 Mar 2025   AIC                         174545.013
Time:                        23:50:18   BIC                         174641.409
Sample:                             0   HQIC                        174575.288
                              - 47250                                         
Covariance Type:                  opg                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.9579      0.102     -9.411      0.000      -1.157      -0.758
temp_hora_1            0.0184      0.002      8.163      0.000       0.014       0.023
temp_hora_2         

/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [49]:
# Variables exógenas
exog_variables = ['humedad_hora_1', 'humedad_hora_2', 'humedad_hora_3', 'humedad_media_general', 'humedad_min_general', 'humedad_max_general']

# Eliminar valores nulos o rellenar
df_pandas_clean = df_pandas.dropna(subset=exog_variables + ['humedad_media'])
X = df_pandas_clean[exog_variables]
y = df_pandas_clean['humedad_media']

# Asegurarse de que los datos sean numéricos
X = X.astype(float)
y = y.astype(float)

# Ajustar el modelo ARIMAX
arimax_model = ARIMA(y, exog=X, order=(2, 0, 1))
arimax_result = arimax_model.fit()

# Resumen del modelo
print(arimax_result.summary())

# Realizar pronósticos (usando las últimas filas de las variables exógenas)
forecast = arimax_result.forecast(steps=10, exog=X.tail(10))
print(forecast)

/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retv

                               SARIMAX Results                                
Dep. Variable:          humedad_media   No. Observations:                47250
Model:                 ARIMA(2, 0, 1)   Log Likelihood             -152233.170
Date:                Sat, 08 Mar 2025   AIC                         304488.340
Time:                        23:50:47   BIC                         304584.735
Sample:                             0   HQIC                        304518.615
                              - 47250                                         
Covariance Type:                  opg                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -5.4325      0.610     -8.908      0.000      -6.628      -4.237
humedad_hora_1            0.0707      0.002     30.736      0.000       0.066       0.075
humedad_

/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
